###  Question- 1

In [0]:
# date conversion, string conversion to int , total sales by month 

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window

data= [

    (1,"delhi","goa",7),
    (2,"mumbai","goa",5),
    (3,"delhi","mumbai",6),
    (4,"mumbai","delhi",6),
    (5,"goa","delhi",7),
    (6,"goa","mumbai",5),
]
schema = StructType ([
    StructField ("route",IntegerType(),True),
    StructField ("source",StringType(),True),
    StructField ("destination",StringType(),True),
    StructField ("time",IntegerType(),True)
    
])
df = spark.createDataFrame(data,schema)

dt = df.select (
        when(col("source") <= col("destination"), col("source")).otherwise(col("destination")).alias("src"),
    when(col("source") <= col("destination"), col("destination")).otherwise(col("source")).alias("dest"),
    col("time")

)

dt.groupBy(col("src"),col("dest")).agg(
    sum(col("time")).alias("tot_time")

).show()



+-----+------+--------+
|  src|  dest|tot_time|
+-----+------+--------+
|delhi|   goa|      14|
|  goa|mumbai|      10|
|delhi|mumbai|      12|
+-----+------+--------+



### Question - 2

In [0]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window
from datetime import datetime

# 1. Define your data using datetime objects
data = [
    (1, "A", "login",  datetime(2025, 6, 25, 15, 30)),
    (2, "B", "login",  datetime(2025, 6, 25, 15, 30)),
    (3, "C", "login",  datetime(2025, 6, 25, 15, 30)),
    (4, "A", "logout", datetime(2025, 6, 25, 16, 30)),
    (5, "B", "logout", datetime(2025, 6, 25, 17, 30)),
    (6, "C", "logout", datetime(2025, 6, 25, 18, 30))
]

# 2. Define schema with TimestampType
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("cust", StringType(), True),
    StructField("status", StringType(), True),
    StructField("date", TimestampType(), True)
])

# 3. Create DataFrame
df = spark.createDataFrame(data, schema)

wind = Window.partitionBy("cust").orderBy("date")

# Add next date column using lead
df = df.withColumn("next_date", lead("date").over(wind))
filt = df.filter(col("next_date").isNotNull())
df_final = filt.withColumn(
    "time_spent_minutes",
    (unix_timestamp("next_date") - unix_timestamp("date")) / 60
)

df_final.show(truncate=False)


+---+----+------+-------------------+-------------------+------------------+
|id |cust|status|date               |next_date          |time_spent_minutes|
+---+----+------+-------------------+-------------------+------------------+
|1  |A   |login |2025-06-25 15:30:00|2025-06-25 16:30:00|60.0              |
|2  |B   |login |2025-06-25 15:30:00|2025-06-25 17:30:00|120.0             |
|3  |C   |login |2025-06-25 15:30:00|2025-06-25 18:30:00|180.0             |
+---+----+------+-------------------+-------------------+------------------+

